In [2]:
from sklearn import cross_validation,model_selection
from sklearn import svm  
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report
from sklearn import cross_validation,model_selection
from sklearn.metrics import precision_recall_fscore_support
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/lil.py:19: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _csparsetools
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:165: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._shortest_path import shortest_path, floyd_warshall, dijkstra,\
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/_validation.py:5: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._tools import csgraph_to_dense, csgraph_from_dense,\
/home/umang/.local/lib/python2.7/site-packages/scipy/sparse/csgraph/__init__.py:167: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._traversal import breadth_first_order, depth_first_order, \
/home/umang/.local/lib/python2.7/site-packages

/home/umang/.local/lib/python2.7/site-packages/pandas/core/groupby/groupby.py:68: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (lib, reduction,
/home/umang/.local/lib/python2.7/site-packages/pandas/core/reshape/reshape.py:30: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos as _algos, reshape as _reshape
/home/umang/.local/lib/python2.7/site-packages/pandas/io/parsers.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  import pandas._libs.parsers as parsers
/home/umang/.local/lib/python2.7/site-packages/pandas/io/pytables.py:50: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib, writers as libwriters
/home/umang/.local/lib/python2.7/site-packages/sklearn/feature_extraction/hashing.py:

In [4]:
word2vec_features = pd.read_csv("../AM_Email_Tagging/Intermediate_Excel_Files/ummang_word2vec.csv")

/home/umang/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
del word2vec_features['Unnamed: 0']
del word2vec_features['Unnamed: 309']

In [6]:
# word2vec_features
opening_tags = ['op_tag','oi_tag','opc_tag','ow_tag','oa_tag','ocg_tag','odf_tag']
closing_tags = ['cac_tag','cap_tag','ccd_tag','cdf_tag','cff_tag','cth_tag','cw_tag']

In [7]:
opening_data = word2vec_features[word2vec_features['Tag_name'].isin(opening_tags)]
closing_data = word2vec_features[word2vec_features['Tag_name'].isin(closing_tags)]

In [8]:
opening_data = opening_data.reset_index(drop='true')
closing_data = closing_data.reset_index(drop='true')

In [9]:
opening_data.columns

Index([u'Amcatid', u'Sentences', u'Tag_name', u'Flow', u'Rater_id',
       u'All_tags', u'cleaned_text', u'0', u'1', u'2',
       ...
       u'291', u'292', u'293', u'294', u'295', u'296', u'297', u'298', u'299',
       u'flag'],
      dtype='object', length=308)

In [10]:
opening_data_X = opening_data.iloc[:,7:307]
opening_data_Y = opening_data.iloc[:,2]
closing_data_X = closing_data.iloc[:,7:307]
closing_data_Y = closing_data.iloc[:,2]

In [ ]:
num_feats = [100,110,120,130,140,150,160,170,180]
models = ['log_reg' , 'svm']
model_list = []
feat_list = []
accuracy_mean_list = []
accuracy_stdev_list = []
report_lr_list = []
for model in models:
    for feat_num in num_feats:
        model_list.append(model)
        feat_list.append(feat_num)
        accuracy_lr = []
        report_lr = []
        print model, feat_num
        for i in tqdm(range(0,20)):
            X_train, X_test, y_train,y_test = cross_validation.train_test_split(opening_data_X,opening_data_Y, test_size = 0.30)
            selectkBest=SelectKBest(f_classif, k=feat_num)

            if model == 'log_reg':
                model_used = LogisticRegression()
            if model == 'svm':
                model_used = svm.SVC()
            model_used.fit(X_train, y_train)
            predictions = model_used.predict(X_test)
            accuracy_lr.append(accuracy_score(y_test, predictions))
            y_pred = predictions
            y_true = y_test
            report_lr.append(precision_recall_fscore_support(y_true, y_pred, average= None , labels=opening_tags))
        report_lr = np.mean(report_lr,axis=0)
        precision_ = list(report_lr[0])
        precision_.append('precision')
        recall_ = list(report_lr[1])
        recall_.append('recall')
        f1_score_ = list(report_lr[2])
        f1_score_.append('f1_score')
        model_type = [model, feat_num]
        precision_.extend(model_type)
        recall_.extend(model_type)
        f1_score_.extend(model_type)
        report_lr_list.append(precision_)
        report_lr_list.append(recall_)
        report_lr_list.append(f1_score_)
        accuracy_mean_list.append(np.mean(accuracy_lr))
        accuracy_stdev_list.append(np.std(accuracy_lr))


  0%|          | 0/20 [00:00<?, ?it/s]

log_reg 100



 75%|███████▌  | 15/20 [01:09<00:23,  4.66s/it]

In [78]:
new_df = pd.DataFrame(report_lr_list,columns=opening_tags+['metric','model','features'])
print new_df
new_df.to_csv('/home/umang/Desktop/AM_Email_Tagging_Model_Building_Results/Gap_of_10:100_to_180/Opening_Tags/classification.csv')

      op_tag    oi_tag   opc_tag    ow_tag    oa_tag   ocg_tag   odf_tag  \
0   0.675432  0.787992  0.715096  0.652657  0.880002  0.649622  0.513795   
1   0.566698  0.903394  0.736553  0.473291  0.898406  0.703693  0.464395   
2   0.615642  0.841490  0.725101  0.533828  0.888876  0.674827  0.487382   
3   0.657253  0.802821  0.721713  0.744564  0.880362  0.653096  0.516465   
4   0.583296  0.900494  0.727996  0.505759  0.904245  0.703243  0.467132   
5   0.617416  0.848667  0.724432  0.595742  0.892030  0.676969  0.490091   
6   0.673448  0.801506  0.721307  0.694236  0.891747  0.656486  0.514746   
7   0.577231  0.899893  0.726234  0.530118  0.901039  0.717246  0.481088   
8   0.621008  0.847770  0.723518  0.588207  0.896126  0.684450  0.496962   
9   0.668560  0.816110  0.725745  0.762086  0.893841  0.644592  0.517671   
10  0.590965  0.897549  0.730409  0.556487  0.897989  0.714087  0.483214   
11  0.626766  0.854684  0.727790  0.622929  0.895546  0.677156  0.499531   
12  0.670593

In [54]:
df = pd.DataFrame()
df['model'] = model_list
df['features'] = feat_list
df['accuracy_mean'] = accuracy_mean_list
df['accuracy_stdev'] = accuracy_stdev_list

In [55]:
df.to_csv('/home/umang/Desktop/AM_Email_Tagging_Model_Building_Results/Gap_of_10:200_to_250/Closing_Tags/accuracy.csv')
df

,model,features,accuracy_mean,accuracy_stdev
0,log_reg,210,0.674202,0.010332
1,log_reg,220,0.681973,0.009710
2,log_reg,230,0.682044,0.009821
3,log_reg,240,0.679063,0.009516
4,svm,210,0.673882,0.008577
5,svm,220,0.675763,0.011654
6,svm,230,0.672818,0.013652
7,svm,240,0.673669,0.010198
